In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pycountry
import plotly.express as px

In [18]:
import pandas as pd

# Step 1: Read all CSV files
df1 = pd.read_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project--II/data/Cleaned data - api in blocks/cleaned_data_first_2000.csv')
df2 = pd.read_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project--II/data/Cleaned data - api in blocks/cleaned_data_first_6000.csv')
df3 = pd.read_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project--II/data/Cleaned data - api in blocks/cleaned_data_first_6991.csv')

# Step 2: Combine last columns
df1['rating'] = df1['rating'].combine_first(df2['rating'])
df1['rating'] = df1['rating'].combine_first(df3['rating'])

# Step 3: Save the DataFrame (if necessary)
df1.to_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project--II/data/cleaned_data_all.csv', index=False)
df = df1

In [30]:
# I forgot to clean the award column, so Im doing it now:
# Remove spaces, lowercase everything, and remove the word 'Michelin'
df['award'] = df['award'].str.strip().str.lower().str.replace('michelin', '')
# Save the cleaned DataFrame back to a CSV file (optional)
df.to_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project--II/data/cleaned_data_all.csv', index=False)

In [31]:
def get_clean_data():
    # File path to your CSV file
    file_path = '/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project--II/data/cleaned_data_all.csv'

    # Try reading the file with a different encoding
    cleaned_data = pd.read_csv(file_path)

    return cleaned_data

df = get_clean_data()
df.head(3) 

,name,address,location,price,cuisine,longitude,latitude,phonenumber,url,websiteurl,award,facilitiesandservices,description,postal_code,city,country,price_convention,rating
0,le palais,"17f, palais de chine hotel, 3, section 1, chen...","taipei, taiwan",$$$$,Cantonese,121.516960,25.049071,8.862218e+11,https://guide.michelin.com/en/taipei-region/ta...,https://www.palaisdechinehotel.com/p/pdc_tw/pa...,3 stars,"Air conditioning,American Express credit card,...","After the departure of the Macanese chef, the ...",103,taipei,taiwan,very expensive,NaN
1,taïrroir,"6f, 299 lequn 3rd road, zhongshan district, ta...","taipei, taiwan",$$$$,Taiwanese contemporary,121.559303,25.082896,8.862850e+11,https://guide.michelin.com/en/taipei-region/ta...,https://www.tairroir.com/,3 stars,"Air conditioning,American Express credit card,...","A portmanteau of Taiwan and terroir, Taïrroir ...",104,taipei,taiwan,very expensive,4.3
2,jl studio,"2f, 689, section 4, yifeng road, nantun distri...","taichung, taiwan",$$$$,"Singaporean, Contemporary",120.628520,24.150486,8.864238e+11,https://guide.michelin.com/en/taichung-region/...,https://jlstudiotw.com/,3 stars,"Air conditioning,American Express credit card,...","JL stands for Jimmy Lim, a Singaporean chef wh...",408,taichung,taiwan,very expensive,4.4


# 1. Geographical Hypothesis

In [32]:
# Data Aggregation

# Count the number of restaurants in each city
restaurants_per_city = df['city'].value_counts()

# Count the number of restaurants in each country
restaurants_per_country = df['country'].value_counts()

# Show the top 10 cities and countries with the most Michelin-rated restaurants
top_cities = restaurants_per_city.head(10)
top_countries = restaurants_per_country.head(10)

#top_cities, top_countries

In [33]:
# Filter out rows with missing latitude or longitude
df_map = df.dropna(subset=['latitude', 'longitude'])

# Create a scatter map using Plotly Express
fig = px.scatter_geo(df_map,
                     lat='latitude',
                     lon='longitude',
                     hover_name='name',
                     size_max=5,
                     title='Geographical Distribution of Michelin-Rated Restaurants',
                     template='plotly',
                     projection='natural earth')

# Center title
fig.update_layout(title_x = 0.5)

fig.show()

In [34]:
# Convert the top cities and countries Series to DataFrames for Plotly
top_cities_df = top_cities.reset_index()
top_cities_df.columns = ['City', 'Number of Restaurants']

top_countries_df = top_countries.reset_index()
top_countries_df.columns = ['Country', 'Number of Restaurants']

# Create a bar chart for the top cities using Plotly Express
fig_cities = px.bar(top_cities_df,
                    x='Number of Restaurants',
                    y='City',
                    title='Top 10 Cities with the Most Michelin-Rated Restaurants',
                    orientation='h',
                    template='plotly_white')

# Center title
fig_cities.update_layout(title_x = 0.5)

fig_cities.show()

In [35]:
# Sort the DataFrame in descending order
top_countries_df_sorted = top_countries_df.sort_values(by='Number of Restaurants', ascending=False)

# Create a bar chart for the top countries using Plotly Express
fig_countries = px.bar(top_countries_df_sorted,
                       x='Number of Restaurants',
                       y='Country',
                       title='Top 10 Countries with the Most Michelin-Rated Restaurants',
                       orientation='h',
                       template='seaborn',  
                       color_discrete_sequence=['#636EFA']  # Set your desired color
                      )

# Center the title
fig_countries.update_layout(title_x=0.5)

fig_countries.show()

In [36]:
# Filter out rows with missing latitude or longitude
df_map = df.dropna(subset=['latitude', 'longitude'])

# Create a density heatmap using Plotly Express
fig = px.density_mapbox(df_map,
                        lat='latitude',
                        lon='longitude',
                        radius=10,
                        center=dict(lat=0, lon=180),
                        zoom=0,
                        mapbox_style="stamen-terrain")

fig.show()

In [39]:
# Group by country and award, and count the number of restaurants
df_grouped = df_stars.groupby(['country', 'award']).size().reset_index(name='count')

# Sort the countries by the total number of Michelin-starred restaurants and get the top 5
top_countries = df_grouped.groupby('country')['count'].sum().sort_values(ascending=False).head(5).index

# Filter the grouped DataFrame to include only the top 10 countries
df_top5 = df_grouped[df_grouped['country'].isin(top_countries)]

# Create a bar chart using a predefined color sequence for the top 10 countries
fig_top5 = px.bar(df_top5, 
                   x='country', 
                   y='count', 
                   color='award', 
                   color_discrete_sequence=px.colors.qualitative.Set1,
                   title='Top 5 Countries with the Most Michelin-Starred Restaurants',
                   template='plotly_white')

# Center the title
fig_top5.update_layout(title_x=0.5)

# Show the plot
fig_top5.show()

In [40]:
# Filter the dataset for "Bib Gourmand" awarded restaurants
df_bib_gourmand = df[df['award'] == 'bib gourmand']

# Group by country and count the number of "Bib Gourmand" restaurants
bib_gourmand_counts = df_bib_gourmand['country'].value_counts().reset_index()
bib_gourmand_counts.columns = ['country', 'count']

# Get the top 5 countries
top5_bib_gourmand = bib_gourmand_counts.head(10)

# Create a bar chart for the top 5 countries
fig_top5_bib = px.bar(top5_bib_gourmand, 
                      x='country', 
                      y='count',
                      title='Top 5 Countries with the Most "Bib Gourmand" Restaurants',
                      template='plotly_white')

# Show the plot
fig_top5_bib.show()

In [41]:
# Filter the dataset for "Green Star" awarded restaurants
df_green_star = df[df['award'] == 'green star']

# Group by country and count the number of "Bib Gourmand" restaurants
green_star_counts = df_green_star['country'].value_counts().reset_index()
green_star_counts.columns = ['country', 'count']

# Get the top 5 countries
top10_green_star = green_star_counts.head(10)

# Create a bar chart for the top 5 countries
fig_top10_green = px.bar(top10_green_star, 
                      x='country', 
                      y='count',
                      title='Top 5 Countries with the Most "Green Star" Restaurants',
                      template='plotly_white')

# Show the plot
fig_top10_green.show()

# 2. Price Range and Ratings Hypothesis

In [43]:
missing_values = df.isnull().sum()
missing_values_percent = (missing_values / len(df))*100
missing_values_df = pd.DataFrame({'Missing Values':missing_values,'Percentage(%)':missing_values_percent})

missing_values_df.sort_values(by='Percentage(%)',ascending=False)

,Missing Values,Percentage(%)
websiteurl,1305,18.664188
rating,1195,17.090961
postal_code,177,2.531465
phonenumber,159,2.274027
price,47,0.672197
price_convention,47,0.672197
facilitiesandservices,46,0.657895
description,11,0.157323
location,0,0.000000
cuisine,0,0.000000


In [44]:

# Redefining the award_mapping dictionary
award_mapping = {'1 star': 1, '2 stars': 2, '3 stars': 3}

# Converting Michelin stars to a numerical format and filling missing values with 0
df['michelin_stars'] = df['award'].map(award_mapping).fillna(0)

# Filling missing values in price_convention with "Not Specified"
df['price_convention'] = df['price_convention'].fillna('Not Specified')

# Optional: Handle missing values in the rating column (if necessary for your analysis)
# data['rating'] = data['rating'].fillna(some_value_or_method)

# Checking for missing values again after the updates
missing_values_selected_columns_updated = df[['price_convention', 'michelin_stars', 'rating']].isnull().sum()

missing_values_selected_columns_updated

price_convention       0
michelin_stars         0
rating              1195
dtype: int64

In [45]:
# Filter out the "Not Specified" values from the price_convention column
data_filtered = df[df['price_convention'] != 'Not Specified']

# Create a boxplot for Google ratings across different price conventions using Plotly, excluding "Not Specified" values
fig_price_filtered = px.box(
    data_filtered, 
    x='price_convention', 
    y='rating',
    title='Distribution of Google Ratings Across Different Price Ranges (Excluding "Not Specified")',
    labels={"price_convention": "Price Convention", "rating": "Google Rating"},
    category_orders={"price_convention": ["Inexpensive", "Moderately", "Expensive", "Very Expensive"]}
)

# Show the plot
fig_price_filtered.show()

In [46]:
# Create a boxplot for Google ratings across different Michelin star categories using Plotly
fig_michelin = px.box(
    df, 
    x='michelin_stars', 
    y='rating',
    title='Distribution of Google Ratings Across Different Michelin Star Categories',
    labels={"michelin_stars": "Number of Michelin Stars", "rating": "Google Rating"},
    category_orders={"michelin_stars": [0, 1, 2, 3]}
)

# Update x-axis tick labels
fig_michelin.update_xaxes(tickvals=[0, 1, 2, 3], ticktext=['0 stars', '1 star', '2 stars', '3 stars'])

# Show the plot
fig_michelin.show()

In [47]:
# Analysing correlation

# Performing a correlation analysis between Michelin stars and Google ratings
correlation_michelin_google = df[['michelin_stars', 'rating']].corr(method='spearman')

# Displaying the correlation coefficient
correlation_michelin_google.iloc[0, 1]

0.2583071933955075

In [48]:
# Data Preparation: Filter the dataset to include only Michelin-rated restaurants
michelin_rated_restaurants = df[df['michelin_stars'] > 0]

# Check for missing values in 'cuisine' and 'rating' columns
missing_values_cuisine_rating = michelin_rated_restaurants[['cuisine', 'rating']].isnull().sum()

# Display the number of Michelin-rated restaurants and missing values
num_michelin_rated_restaurants = michelin_rated_restaurants.shape[0]
missing_values_cuisine_rating, num_michelin_rated_restaurants


(cuisine      0
 rating     557
 dtype: int64,
 3446)

In [49]:
# Exclude restaurants with missing Google ratings from the analysis
michelin_rated_restaurants_filtered = michelin_rated_restaurants.dropna(subset=['rating'])

# Identify Popular Cuisines: Calculate the frequency of each cuisine type among Michelin-rated restaurants
cuisine_frequency = michelin_rated_restaurants_filtered['cuisine'].value_counts()

# Display the top 10 most popular cuisines among Michelin-rated restaurants
top_cuisines = cuisine_frequency.head(10)
top_cuisines


cuisine
Modern Cuisine              580
Creative                    353
Japanese                    137
Creative, Modern Cuisine     85
Contemporary                 62
Modern Cuisine, Creative     59
Modern French                58
Classic Cuisine              55
French                       49
Sushi                        46
Name: count, dtype: int64

In [50]:
df['cuisine'].unique()

array(['Cantonese', 'Taiwanese contemporary', 'Singaporean, Contemporary',
       'Contemporary, French', 'Contemporary, Californian',
       'Asian, Contemporary', 'European Contemporary', 'French',
       'French Contemporary', 'Creative', 'Modern Cuisine, Creative',
       'Modern Cuisine', 'Creative, Contemporary', 'Innovative', 'Sushi',
       'Italian', 'Creative, Modern Cuisine', 'Classic French, Creative',
       'Classic French', 'Modern French, Creative',
       'Creative, International', 'Modern French', 'Creative British',
       'Modern British', 'Creative French', 'Creative, Regional Cuisine',
       'Creative, Seafood', 'Creative, Mediterranean Cuisine',
       'Classic Cuisine', 'Modern Cuisine, Classic Cuisine',
       'Mediterranean Cuisine, Modern Cuisine', 'Seafood', 'Vegetarian',
       'Taizhou', 'Creative, Traditional Cuisine', 'Creative, Innovative',
       'French, Contemporary', 'Japanese', 'Chinese',
       'Italian Contemporary', 'Mediterranean Cuisine, Cont

In [51]:
# Step 1: Text Standardization
df['cuisine_cleaned'] = df['cuisine'].str.lower().str.strip()
df[['cuisine', 'cuisine_cleaned']].head()


,cuisine,cuisine_cleaned
0,Cantonese,cantonese
1,Taiwanese contemporary,taiwanese contemporary
2,"Singaporean, Contemporary","singaporean, contemporary"
3,"Contemporary, French","contemporary, french"
4,"Contemporary, Californian","contemporary, californian"


In [52]:
def filter_cuisine(cuisine):
    # Keywords to exclude
    exclude_keywords = ["contemporary", "creative", "modern cuisine"]
    
    # Split the cuisine string by commas
    cuisines = cuisine.split(',')
    
    # Keep the first value that doesn’t include the exclude keywords
    for c in cuisines:
        c = c.strip()
        if not any(keyword in c for keyword in exclude_keywords):
            return c
    
    # If none of the values meet the criteria, return the original string
    return cuisine

# Apply the filter function to the 'cuisine_cleaned' column
df['cuisine_filtered'] = df['cuisine_cleaned'].apply(filter_cuisine)

# Display the original and cleaned 'cuisine' columns
df[['cuisine', 'cuisine_cleaned', 'cuisine_filtered']].head(10)

,cuisine,cuisine_cleaned,cuisine_filtered
0,Cantonese,cantonese,cantonese
1,Taiwanese contemporary,taiwanese contemporary,taiwanese contemporary
2,"Singaporean, Contemporary","singaporean, contemporary",singaporean
3,"Contemporary, French","contemporary, french",french
4,"Contemporary, Californian","contemporary, californian",californian
5,"Contemporary, Californian","contemporary, californian",californian
6,"Contemporary, Californian","contemporary, californian",californian
7,"Asian, Contemporary","asian, contemporary",asian
8,"Contemporary, French","contemporary, french",french
9,European Contemporary,european contemporary,european contemporary


In [53]:
df['cuisine_filtered'].nunique()

268

In [54]:
def clean_cuisine(cuisine):
    # Step 1: Text Standardization
    cuisine = cuisine.lower().strip()
    
    # Step 2: Filtering Based on Keywords
    keywords = ["contemporary", "creative", "modern cuisine"]
    if ',' in cuisine:
        # Split the string by commas
        values = cuisine.split(',')
        # Keep the first value that includes one of the keywords
        for value in values:
            if any(keyword in value for keyword in keywords):
                return value.strip()
        # If none of the keywords are found, keep the first value
        return values[0].strip()
    else:
        return cuisine

# Apply the cleaning function to the 'cuisine' column
df['cleaned_cuisine'] = df['cuisine'].astype(str).apply(clean_cuisine)

# Display the first few rows of the original and cleaned 'cuisine' columns
df[['cuisine', 'cleaned_cuisine']].head(5)


,cuisine,cleaned_cuisine
0,Cantonese,cantonese
1,Taiwanese contemporary,taiwanese contemporary
2,"Singaporean, Contemporary",contemporary
3,"Contemporary, French",contemporary
4,"Contemporary, Californian",contemporary


In [55]:
df['cleaned_cuisine'].nunique()

229

In [56]:
# Get the top 5 most frequent values in the 'cleaned_cuisine' column
top_5_cuisines = df['cleaned_cuisine'].value_counts().head(5)
top_5_cuisines


cleaned_cuisine
modern cuisine         1390
creative                753
contemporary            411
japanese                354
traditional cuisine     254
Name: count, dtype: int64

In [57]:
import plotly.express as px

# Assuming 'top_5_cuisines' is already defined in your local environment
fig = px.bar(top_5_cuisines, 
             title='Top 5 Cuisines', 
             labels={'index': 'Cuisine', 'value': 'Frequency'},
             text=top_5_cuisines.values,
             template="plotly_white")

# Update the layout of the plot
fig.update_layout(
    font=dict(size=14),
    title=dict(x=0.5, xanchor='center'),
    xaxis=dict(title='Cuisine'),
    yaxis=dict(title='Frequency')
)

# Show the plot
fig.show()